# 3.5 Colocando o Istio em execução

## Instalando o Istio (linux)

Para instalar a última versão do Istio, neste momento 1.8.0, você pode ir até a página [Getting Started](https://istio.io/latest/docs/setup/getting-started/#download) ou seguir as instruções abaixo:

In [ ]:
curl -L https://istio.io/downloadIstio | sh -

Para usar o comando `istioctl`, que está no diretório `bin` do download, coloque-o na variável `PATH` ou copie o arquivo `bin/istioctl` para um diretório no seu `PATH`

## Opção 1: Copiar o arquivo para seu diretório de binários

In [ ]:
# Verificar onde está o executável do Istio
ls istio-1.8.0/bin
# Verificar o conteúdo da variável PATH
echo $PATH
# Vamos coloca-lo no diretório bin do usuário
[ -d "~/bin" ] && echo "O diretorio $HOME/bin existe."
# Copiar o arquivo para o diretório bin
cp istio-1.8.0/bin/istioctl ~/bin
# Adicionar o diretório de binários do usuário no path
export PATH=~/bin:$PATH

In [ ]:
# Teste
istioctl version

## Opção 2: Definir a variável PATH
Você pode copiar o comando em qualquer diretório ou mantê-lo no diretório do Istio e ajustar a variável `PATH` na sessão.

In [ ]:
export PATH=$PWD/istio-1.8.0/bin:$PATH
echo $PATH
istioctl --help

## Acessando o k8s

Você já deve ter acessado o cluster em seções anteriores, mas vale a pena revisar:

In [ ]:
# Se você não estiver usando o docker-desktop será necessário obter o arquivo de configuração e ajustar a variável KUBECONFIG
#export KUBECONFIG=~/.kube/config
kubectl config get-contexts
kubectl get nodes

## O mínimo do Istio

Desde a versão 1.6 o Istio é composto de uma única entrega chamada `istiod` e ela pode ser instalada com o comando abaix:

In [ ]:
istioctl install --set profile=minimal --skip-confirmation  

Vamos verificar o que foi instalado

In [ ]:
# Obtendo os namespaces
kubectl get ns

# Verificando o namespace do Istio
kubectl get all -n istio-system

## Ativando o Istio em um namespace
Para que o Istio injete um proxy no mesmo pod que o(s) seu(s) containers, você precisa marcar o namespace da seguinte forma:

In [ ]:
kubectl label namespace default istio-injection=enabled

Vamos verificar como ficou a configuração do namespace

In [ ]:
kubectl describe ns/default

O `istiod` irá monitorar os namespaces com esse `label` e adicionar ao pod um container de proxy. Vamos fazer deploy de uma aplicação exemplo para verificar esse comportamento.

## Criando uma aplicação demo

Vamos utilizar o helm para criar uma aplicação simples.

In [ ]:
# Vamos criar um diretório para nossos exemplos
mkdir -p exemplos/simple-app

# Deployment
cat <<EOT > exemplos/2_simple-app/deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: simple-app
  labels:
    app: simple-app
    version: v1
spec:
  replicas: 1
  selector:
    matchLabels:
        app: simple-app
        version: v1
  template:
    metadata:
      labels:
        app: simple-app
        version: v1
    spec:
      containers:
        - name: simple-app
          image: "nginx:stable"
          imagePullPolicy: IfNotPresent
          ports:
            - name: http
              containerPort: 80
              protocol: TCP
EOT

# Service
cat <<EOT > exemplos/2_simple-app/service.yaml
apiVersion: v1
kind: Service
metadata:
  name: simple-app
  labels:
    app: simple-app
    version: v1
spec:
  type: ClusterIP
  ports:
    - port: 80
      targetPort: http
      protocol: TCP
      name: http
  selector:
    app: simple-app
    version: v1
EOT

In [ ]:
ls -la exemplos/2_simple-app

Criamos dois arquivos, o `deployment.yaml` e o `service.yaml` no diretório `exemplos/simple-app`, agora vamos instala-la no cluster.

Inspecione os arquivos e tente descobrir o que será instalado no cluster, uma dica, procure a pela imagem.

In [ ]:
kubectl apply -f exemplos/2_simple-app

Verificando a situação do POD.

In [ ]:
kubectl get pods

Vamos acessar nossa aplicação, ela foi configurada para o tipo de serviço `ClusterIP`, o que significa que o acesso é interno, apenas entre os PODs do cluster, mas podemos acessá-la utilizando o comando `kubectl port-forwar`.

In [ ]:
kubectl port-forward svc/simple-app 8000:80

Vá até o seu navegador e entre acesse a url: <http://localhost:8000>

Pronto, você tem acesso a sua aplicação como se estivesse sendo executada na sua máquina. Claro que o kubernetes está na sua máquina, mas isso funcionaria em qualquer kubernetes, local ou remoto.

Para interromper:
* No jupyter Lab ou notebook: Clique no icone <kbd>◾</kbd> (_Interrup the kernel_ na barra de ferramentas)
* No terminal: tecle <kbd>CTRL</kbd>+<kbd>C</kbd>

Verificando o que foi instalado.

In [ ]:
kubectl get --show-labels all

Note que o pod/simple-app tem 2/2 na coluna pronto (Ready), isso significa que dois containers de dois estão ok, vamos verificar quem é o segundo container.

In [ ]:
# Usaremos um dos labels do pod para encontra-lo
kubectl describe pod -l app=simple-app

É uma grande quantidade de informação, vamos procurar uma seção chamada `Containers` e nela a nossa aplicação `simple-app`.

Como você pode ver, a imagem desse container é `nginx`, com a tag `stable`, mais abaixo tem um segundo container `istio-proxy`, com a imagem `docker.io/istio/proxyv2` e tag representando a versão `1.8.0`.

Esse container não faz parte do [exemplos/simple-app/deployment.yaml](exemplos/simple-app/deployment.yaml), ele foi adicionado ao seu pod pelo `istiod`.

Caso você precise saber todos os _namespaces_ que tem a injeção do _conteiner_ _proxy_ do Istio ativado, basta executar o comando:

In [ ]:
kubectl get ns -l istio-injection=enabled

Até o momento, somente o namespace `default`, onde configuramos a marcação, retornou.

## Conclusão

Neste ponto temos o mínimo do Istio em execução no nosso cluster, mas com exceção de um container extra isso não significa muito.

Na próxima parte iremos explorar os recursos que esta instalação mínima do Istio pode oferecer.